# Chapter 7 - How to prepare the data

In [1]:
import pandas as pd

## How to work with datetime columns

In [2]:
fires = pd.read_pickle('fires_cleaned.pkl')
fires.head()

fire_name  fire_year state discovery_date contain_date  acres_burned
16         POWER       2004    CA     2004-10-06   2004-10-21       16823.0
17         FREDS       2004    CA     2004-10-13   2004-10-17        7700.0
25      BACHELOR       2004    NM     2004-07-20   2004-07-20          10.0
37    HOWARD GAP       2005    NC     2005-01-27   2005-01-28          50.3
39  AUSTIN CREEK       2005    NC     2005-02-12   2005-02-13         125.0

In [3]:
fires['fire_month'] = fires.discovery_date.dt.month

In [4]:
fires['days_burning'] = (fires.contain_date - fires.discovery_date).dt.days

In [5]:
fires.head()

fire_name  fire_year state discovery_date contain_date  acres_burned  \
16         POWER       2004    CA     2004-10-06   2004-10-21       16823.0   
17         FREDS       2004    CA     2004-10-13   2004-10-17        7700.0   
25      BACHELOR       2004    NM     2004-07-20   2004-07-20          10.0   
37    HOWARD GAP       2005    NC     2005-01-27   2005-01-28          50.3   
39  AUSTIN CREEK       2005    NC     2005-02-12   2005-02-13         125.0   

    fire_month  days_burning  
16          10          15.0  
17          10           4.0  
25           7           0.0  
37           1           1.0  
39           2           1.0

## How to work with string and numeric columns

In [6]:
fires['fire_name'] = fires.fire_name.str.title()

In [7]:
fires['full_name'] = 'The ' + fires.fire_name + ' Fire ' \
                   + '(' + fires.fire_year.astype(str) + ')'

In [8]:
fires['acres_per_day'] = fires.dropna().acres_burned / fires.dropna().days_burning

In [9]:
fires[['fire_name','full_name','acres_burned','days_burning','acres_per_day']].head()

fire_name                     full_name  acres_burned  days_burning  \
16         Power         The Power Fire (2004)       16823.0          15.0   
17         Freds         The Freds Fire (2004)        7700.0           4.0   
25      Bachelor      The Bachelor Fire (2004)          10.0           0.0   
37    Howard Gap    The Howard Gap Fire (2005)          50.3           1.0   
39  Austin Creek  The Austin Creek Fire (2005)         125.0           1.0   

    acres_per_day  
16    1121.533333  
17    1925.000000  
25            inf  
37      50.300000  
39     125.000000

## How to add summary columns

In [10]:
fires[['state','days_burning']].head()

state  days_burning
16    CA          15.0
17    CA           4.0
25    NM           0.0
37    NC           1.0
39    NC           1.0

In [11]:
fires['mean_days'] = fires.groupby('state')['days_burning'].transform(func='mean')
fires[['state','days_burning','mean_days']].head()

state  days_burning  mean_days
16    CA          15.0   5.387197
17    CA           4.0   5.387197
25    NM           0.0   6.085806
37    NC           1.0   1.015474
39    NC           1.0   1.015474

## How to apply functions to rows or columns

In [12]:
workData = pd.read_pickle('workData.pkl')
workData.head(3)

sex  region  wrkstat  hrs1  wkcontct  talkspvs  effctsup
id                                                          
2     2       1      1.0  40.0       3.0       4.0       4.0
4     2       1      2.0  20.0       1.0       4.0       4.0
14    2       2      1.0  37.0       1.0       4.0       3.0

In [13]:
workData.apply('mean')

sex          1.529897
region       5.184536
wrkstat      1.163918
hrs1        42.083505
wkcontct     2.796907
talkspvs     3.291753
effctsup     3.253608
dtype: float64

In [14]:
import numpy as np
workData[['sex','hrs1']].apply(np.mean)

sex      1.529897
hrs1    42.083505
dtype: float64

In [15]:
workData['avg_rating'] = workData[
    ['wkcontct','talkspvs','effctsup']].apply(np.mean, axis=1)
workData.head(3)

sex  region  wrkstat  hrs1  wkcontct  talkspvs  effctsup  avg_rating
id                                                                      
2     2       1      1.0  40.0       3.0       4.0       4.0    3.666667
4     2       1      2.0  20.0       1.0       4.0       4.0    3.000000
14    2       2      1.0  37.0       1.0       4.0       3.0    2.666667

## How to apply user-defined functions

In [16]:
def convert_sex(row):
    if row.sex == 1:
        return 'male'
    elif row.sex == 2:
        return 'female'
    else:
        return 'non-binary'
    
workData['sex'] = workData.apply(convert_sex, axis=1)
workData.head()

sex  region  wrkstat  hrs1  wkcontct  talkspvs  effctsup  avg_rating
id                                                                         
2   female       1      1.0  40.0       3.0       4.0       4.0    3.666667
4   female       1      2.0  20.0       1.0       4.0       4.0    3.000000
14  female       2      1.0  37.0       1.0       4.0       3.0    2.666667
19    male       1      1.0  50.0       1.0       3.0       4.0    2.666667
21  female       1      1.0  38.0       1.0       4.0       4.0    3.000000

In [17]:
# get the data
gameData = pd.read_pickle('shot_cleaned.pkl')[['game_id','game_date']]
gameData[gameData.duplicated(keep=False)]
gameData.drop_duplicates(keep='first', inplace=True)
gameData

game_id  game_date
0      0020900015 2009-10-28
12     0020900030 2009-10-30
21     0020900069 2009-11-04
27     0020900082 2009-11-06
32     0020900096 2009-11-08
...           ...        ...
11776  0021801164 2019-04-02
11787  0021801179 2019-04-04
11801  0021801191 2019-04-05
11822  0021801205 2019-04-07
11842  0021801215 2019-04-09

[692 rows x 2 columns]

In [18]:
def get_season(row):
    if row.game_date.month > 6:
        season = f'{row.game_date.year}-{row.game_date.year + 1}'
    else:
        season = f'{row.game_date.year - 1}-{row.game_date.year}'
    return season

gameData['season'] = gameData.apply(get_season, axis=1)
with pd.option_context('display.max_rows', 6, 'display.max_columns', None):
    display(gameData)

game_id  game_date     season
0      0020900015 2009-10-28  2009-2010
12     0020900030 2009-10-30  2009-2010
21     0020900069 2009-11-04  2009-2010
...           ...        ...        ...
11801  0021801191 2019-04-05  2018-2019
11822  0021801205 2019-04-07  2018-2019
11842  0021801215 2019-04-09  2018-2019

[692 rows x 3 columns]

## How lambda expressions work with DataFrames

In [19]:
df = pd.DataFrame([[0,1,2],[3,4,5]], columns=['col1','col2','col3'])
df

col1  col2  col3
0     0     1     2
1     3     4     5

In [20]:
df.apply(lambda x: x.sum() * 2, axis=0)

col1     6
col2    10
col3    14
dtype: int64

In [21]:
df.apply(lambda x: x.sum() * 2, axis=1)

0     6
1    24
dtype: int64

## How to apply lambda expressions

In [22]:
workData['wrkstat'] = workData.apply(
    lambda row: 'full-time' if row.wrkstat == 1.0 else 'part-time', axis=1)
workData.head()

sex  region    wrkstat  hrs1  wkcontct  talkspvs  effctsup  avg_rating
id                                                                           
2   female       1  full-time  40.0       3.0       4.0       4.0    3.666667
4   female       1  part-time  20.0       1.0       4.0       4.0    3.000000
14  female       2  full-time  37.0       1.0       4.0       3.0    2.666667
19    male       1  full-time  50.0       1.0       3.0       4.0    2.666667
21  female       1  full-time  38.0       1.0       4.0       4.0    3.000000

In [23]:
carsData = pd.read_csv('cars.csv')

carsData['Brand'] = carsData.apply(lambda x: x.CarName.split()[0], axis=1)
carsData[['CarName','Brand']].head()

CarName        Brand
0        alfa-romero giulia  alfa-romero
1       alfa-romero stelvio  alfa-romero
2  alfa-romero Quadrifoglio  alfa-romero
3               audi 100 ls         audi
4                audi 100ls         audi

## How to set or remove an index

In [24]:
fires_by_month = pd.read_pickle('fires_by_month.pkl')

In [25]:
fires_by_month.set_index('state', inplace=True)
fires_by_month.head(3)

fire_year  fire_month  acres_burned  days_burning  fire_count
state                                                               
AK          1992           5        4202.0         135.0          14
AK          1992           6       86401.0         417.0          23
AK          1992           7       48516.7         500.0          26

In [26]:
fires_by_month = pd.read_pickle('fires_by_month.pkl')

In [27]:
fires_by_month.set_index(['state','fire_year','fire_month'], inplace=True)
fires_by_month.head(3)

acres_burned  days_burning  fire_count
state fire_year fire_month                                        
AK    1992      5                 4202.0         135.0          14
                6                86401.0         417.0          23
                7                48516.7         500.0          26

In [28]:
fires_no_index = fires_by_month.reset_index()
fires_no_index.head(3)

state  fire_year  fire_month  acres_burned  days_burning  fire_count
0    AK       1992           5        4202.0         135.0          14
1    AK       1992           6       86401.0         417.0          23
2    AK       1992           7       48516.7         500.0          26

## How to unstack indexed data

In [29]:
# get indexed dataset
top5_states = pd.read_pickle('top_states.pkl')
top5_states.head(3)

acres_burned  days_burning  fire_count
state fire_year                                        
AK    1992           142444.7        1145.0        68.0
      1993           686630.5        3373.0       144.0
      1994           261604.7        2517.0       126.0

In [30]:
# unstack the state level
top_wide = top5_states[['days_burning','fire_count']].unstack(level='state')
# top_wide = top5_states[['days_burning','fire_count']].unstack(level=0)
top_wide.head(3)

days_burning                             fire_count                \
state               AK      ID     CA    TX     NV         AK     ID     CA   
fire_year                                                                     
1992            1145.0  1375.0  434.0  11.0   88.0       68.0  192.0  819.0   
1993            3373.0   130.0  302.0  39.0   83.0      144.0   33.0  726.0   
1994            2517.0  3039.0  727.0  35.0  235.0      126.0  245.0  720.0   

                        
state        TX     NV  
fire_year               
1992       22.0   65.0  
1993       42.0   62.0  
1994       54.0  109.0

In [31]:
top_wide = top5_states.unstack(level='state')
top_wide.head(3)

acres_burned                                         days_burning  \
state               AK        ID        CA        TX        NV           AK   
fire_year                                                                     
1992          142444.7  683495.2  289254.9   31500.3   92085.1       1145.0   
1993          686630.5    7658.5  315011.1  114265.5   53733.0       3373.0   
1994          261604.7  727133.0  401619.2   69200.2  195645.6       2517.0   

                                      fire_count                             
state          ID     CA    TX     NV         AK     ID     CA    TX     NV  
fire_year                                                                    
1992       1375.0  434.0  11.0   88.0       68.0  192.0  819.0  22.0   65.0  
1993        130.0  302.0  39.0   83.0      144.0   33.0  726.0  42.0   62.0  
1994       3039.0  727.0  35.0  235.0      126.0  245.0  720.0  54.0  109.0

In [32]:
top_wide = top5_states.fire_count.unstack(level='state')
top_wide.head(3)

state         AK     ID     CA    TX     NV
fire_year                                  
1992        68.0  192.0  819.0  22.0   65.0
1993       144.0   33.0  726.0  42.0   62.0
1994       126.0  245.0  720.0  54.0  109.0

## How to join DataFrames

In [33]:
# get the shots DataFrame
allShotData = pd.read_pickle('shot_cleaned.pkl')
shots = allShotData.drop(columns=['period','minutes_remaining',
                                  'seconds_remaining','loc_x','loc_y','home_team',
                                  'game_date','shot_attempted_flag','shot_made_flag',
                                  'action_type','visiting_team'])
shots1 = shots.head(2)
shots2 = shots.query('game_id == "0020900030"').head(1)
shots3 = shots.query('game_id == "0020900069"').head(1)
shots = pd.concat([shots1,shots2,shots3], ignore_index=True)
shots.set_index('game_id', inplace = True)
shots

player_name   event_type       shot_type  shot_distance
game_id                                                              
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18
0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24
0020900069  Stephen Curry    Made Shot  3PT Field Goal             25

In [34]:
# get the points_by_game DataFrame
points_by_game = pd.read_pickle('pointsScoredGame.pkl')
points_by_game = points_by_game.query('game_id == "0020900015" or game_id == "0020900030" or game_id == "0020900082"')
points_by_game

total_score
game_id                
0020900015           14
0020900030           12
0020900082            2

In [35]:
shots_joined = shots.join(points_by_game, how='inner')
shots_joined

player_name   event_type       shot_type  shot_distance  \
game_id                                                                 
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26   
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18   
0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24   

            total_score  
game_id                  
0020900015           14  
0020900015           14  
0020900030           12

In [36]:
points_by_game2 = points_by_game.copy(deep='true')
points_by_game2['player_name'] = 'Steph Curry'
points_by_game2

total_score  player_name
game_id                             
0020900015           14  Steph Curry
0020900030           12  Steph Curry
0020900082            2  Steph Curry

In [37]:
shots_joined = shots.join(points_by_game2, lsuffix='_1', rsuffix='_2', 
                          how='left')
shots_joined

player_name_1   event_type       shot_type  shot_distance  \
game_id                                                                 
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26   
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18   
0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24   
0020900069  Stephen Curry    Made Shot  3PT Field Goal             25   

            total_score player_name_2  
game_id                                
0020900015         14.0   Steph Curry  
0020900015         14.0   Steph Curry  
0020900030         12.0   Steph Curry  
0020900069          NaN           NaN

In [38]:
shots_joined_outer = shots.join(points_by_game2, lsuffix='_1', 
                                rsuffix='_2', how='outer')
shots_joined_outer

player_name_1   event_type       shot_type  shot_distance  \
game_id                                                                 
0020900015  Stephen Curry  Missed Shot  3PT Field Goal           26.0   
0020900015  Stephen Curry    Made Shot  2PT Field Goal           18.0   
0020900030  Stephen Curry  Missed Shot  3PT Field Goal           24.0   
0020900069  Stephen Curry    Made Shot  3PT Field Goal           25.0   
0020900082            NaN          NaN             NaN            NaN   

            total_score player_name_2  
game_id                                
0020900015         14.0   Steph Curry  
0020900015         14.0   Steph Curry  
0020900030         12.0   Steph Curry  
0020900069          NaN           NaN  
0020900082          2.0   Steph Curry

## Merge

In [39]:
shots2 = shots.reset_index()
shots2

game_id    player_name   event_type       shot_type  shot_distance
0  0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26
1  0020900015  Stephen Curry    Made Shot  2PT Field Goal             18
2  0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24
3  0020900069  Stephen Curry    Made Shot  3PT Field Goal             25

In [40]:
points_by_game2 = points_by_game.reset_index()
points_by_game2

game_id  total_score
0  0020900015           14
1  0020900030           12
2  0020900082            2

In [41]:
shots_merged = shots2.merge(points_by_game2, on='game_id', how='left')
shots_merged

game_id    player_name   event_type       shot_type  shot_distance  \
0  0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26   
1  0020900015  Stephen Curry    Made Shot  2PT Field Goal             18   
2  0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24   
3  0020900069  Stephen Curry    Made Shot  3PT Field Goal             25   

   total_score  
0         14.0  
1         14.0  
2         12.0  
3          NaN

## Concat

In [42]:
# get the data
top5_fires = fires.sort_values('acres_burned', ascending=False).head(5)
top5_fires = top5_fires.reset_index(drop=True)
top5_fires.head()

fire_name  fire_year state discovery_date contain_date  \
0             Inowak       1997    AK     1997-06-25   1997-09-09   
1          Long Draw       2012    OR     2012-07-08   2012-07-30   
2             Wallow       2011    AZ     2011-05-29   2011-07-12   
3           Boundary       2004    AK     2004-06-13   2004-09-30   
4  Minto Flats South       2009    AK     2009-06-21   2009-09-11   

   acres_burned  fire_month  days_burning                          full_name  \
0      606945.0           6          76.0             The Inowak Fire (1997)   
1      558198.3           7          22.0          The Long Draw Fire (2012)   
2      538049.0           5          44.0             The Wallow Fire (2011)   
3      537627.0           6         109.0           The Boundary Fire (2004)   
4      517078.0           6          82.0  The Minto Flats South Fire (2009)   

   acres_per_day  mean_days  
0    7986.118421  32.081535  
1   25372.650000   8.310818  
2   12228.386364   4.887679  
3    4932.357798  32.081535  
4    6305.829268  32.081535

In [43]:
fires_1 = top5_fires.iloc[:3]
fires_1

fire_name  fire_year state discovery_date contain_date  acres_burned  \
0     Inowak       1997    AK     1997-06-25   1997-09-09      606945.0   
1  Long Draw       2012    OR     2012-07-08   2012-07-30      558198.3   
2     Wallow       2011    AZ     2011-05-29   2011-07-12      538049.0   

   fire_month  days_burning                  full_name  acres_per_day  \
0           6          76.0     The Inowak Fire (1997)    7986.118421   
1           7          22.0  The Long Draw Fire (2012)   25372.650000   
2           5          44.0     The Wallow Fire (2011)   12228.386364   

   mean_days  
0  32.081535  
1   8.310818  
2   4.887679

In [44]:
fires_2 = top5_fires.iloc[3:]
fires_2.reset_index(drop=True, inplace=True)
fires_2 = fires_2.drop(columns=['fire_month','days_burning'])
fires_2

fire_name  fire_year state discovery_date contain_date  \
0           Boundary       2004    AK     2004-06-13   2004-09-30   
1  Minto Flats South       2009    AK     2009-06-21   2009-09-11   

   acres_burned                          full_name  acres_per_day  mean_days  
0      537627.0           The Boundary Fire (2004)    4932.357798  32.081535  
1      517078.0  The Minto Flats South Fire (2009)    6305.829268  32.081535

In [45]:
fires_concat = pd.concat([fires_1,fires_2], ignore_index=True)
fires_concat.head(10)

fire_name  fire_year state discovery_date contain_date  \
0             Inowak       1997    AK     1997-06-25   1997-09-09   
1          Long Draw       2012    OR     2012-07-08   2012-07-30   
2             Wallow       2011    AZ     2011-05-29   2011-07-12   
3           Boundary       2004    AK     2004-06-13   2004-09-30   
4  Minto Flats South       2009    AK     2009-06-21   2009-09-11   

   acres_burned  fire_month  days_burning                          full_name  \
0      606945.0         6.0          76.0             The Inowak Fire (1997)   
1      558198.3         7.0          22.0          The Long Draw Fire (2012)   
2      538049.0         5.0          44.0             The Wallow Fire (2011)   
3      537627.0         NaN           NaN           The Boundary Fire (2004)   
4      517078.0         NaN           NaN  The Minto Flats South Fire (2009)   

   acres_per_day  mean_days  
0    7986.118421  32.081535  
1   25372.650000   8.310818  
2   12228.386364   4.887679  
3    4932.357798  32.081535  
4    6305.829268  32.081535

## What the SettingWithCopyWarning is warning you about

In [46]:
df = shots.copy(deep=True)

In [47]:
df.head(3)

player_name   event_type       shot_type  shot_distance
game_id                                                              
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18
0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24

In [48]:
dfSlice = df.loc['0020900015',:]
dfSlice.loc[:,'player_name'] = 'Curry'

C:\Users\Joel\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [49]:
df.head(3)

player_name   event_type       shot_type  shot_distance
game_id                                                              
0020900015          Curry  Missed Shot  3PT Field Goal             26
0020900015          Curry    Made Shot  2PT Field Goal             18
0020900030  Stephen Curry  Missed Shot  3PT Field Goal             24

In [50]:
dfSlice.head(3)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

## When the SettingWithCopyWarning is given

### Generates the warning but no corruption

In [51]:
df = shots.copy(deep=True)

In [52]:
dfSlice = df.query('game_id == "0020900015"')
dfSlice.loc[:,'player_name'] = 'Curry'
df.head(2)

C:\Users\Joel\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


player_name   event_type       shot_type  shot_distance
game_id                                                              
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18

In [53]:
dfSlice.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

#### How use the copy() method to stop the warning message

In [54]:
dfFixed = df.query('game_id == "0020900015"').copy()
dfFixed.loc[:,'player_name'] = 'Curry'

### Generates the warning and corrupts the data

In [55]:
df = shots.copy(deep=True)

In [56]:
dfSlice = df.loc['0020900015',:]
dfSlice.loc[:,'player_name'] = 'Curry'
df.head(2)

C:\Users\Joel\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

In [57]:
dfSlice.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

#### How to fix this code

In [58]:
df = shots.copy(deep=True)

In [59]:
dfFixed = df.loc['0020900015',:].copy()
dfFixed.loc[:,'player_name'] = 'Curry'
df.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                              
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18

In [60]:
dfFixed.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

## When the SettingWithCopyWarning isn’t given

In [61]:
df = shots.copy(deep=True)

In [62]:
dfSlice = df
dfSlice.loc[:,'player_name'] = 'Curry'
df.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

In [63]:
dfSlice.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18

### How to fix this code

In [64]:
df = shots.copy(deep=True)

In [65]:
dfFixed = df.copy()
dfFixed.loc[:,'player_name'] = 'Curry'
df.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                              
0020900015  Stephen Curry  Missed Shot  3PT Field Goal             26
0020900015  Stephen Curry    Made Shot  2PT Field Goal             18

In [66]:
dfFixed.head(2)

player_name   event_type       shot_type  shot_distance
game_id                                                           
0020900015       Curry  Missed Shot  3PT Field Goal             26
0020900015       Curry    Made Shot  2PT Field Goal             18